In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import numpy as np
from copy import deepcopy # 모델 파라미터값 깊은 복사 (베스트 모델 저장을 위함)
from torch.utils.data import TensorDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train= pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
display(train)

In [ ]:
train.shape

In [ ]:
X_train= train.drop(['label'],axis= 1).values
y_train=train['label'].values
X_test=test

In [ ]:
print(X_train.shape, y_train.shape)
#x_train은 2차원 텐서 , y_train은 벡터

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
#픽셀 28*28 , 흑백이기 때문에 한 개의 채널
X_train=X_train.reshape(-1,1,28,28)
X_train.shape

In [ ]:
import matplotlib.pyplot as plt
image_index = 2
image = X_train[image_index, 0]  # 42000개 중 두 번째 이미지 선택, 채널 차원 제거
plt.imshow(image, cmap='gray')  # 흑백 이미지로 표시
plt.show()


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle=True)

In [ ]:
X_train.shape

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)  
y_train = torch.tensor(y_train, dtype=torch.long)     
X_val = torch.tensor(X_val, dtype=torch.float32)     
y_val = torch.tensor(y_val, dtype=torch.long)         

In [ ]:
train_dataset=TensorDataset(X_train, y_train)
train_batches=DataLoader(train_dataset,batch_size=128,shuffle=True)

val_dataset=TensorDataset(X_val, y_val)
val_batches=DataLoader(val_dataset,batch_size=128, shuffle=True)

In [ ]:
X_train.shape

In [ ]:
m=nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1,bias=False)
print(m(X_train).shape)

In [ ]:
class CNNModel(nn.Module):
        
    def __init__(self):
        super().__init__()
        
        self.conv_layers = nn.Sequential(
            
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(32),            
            nn.AvgPool2d(2),
                
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(64),            
            nn.AvgPool2d(2),
               
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.1),
            nn.BatchNorm2d(128),            
            nn.AvgPool2d(2)
        )
            
        self.linear_layers = nn.Sequential(
            nn.Linear(128*3*3, 64),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(64),
                
            nn.Linear(64, 32),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(32),
                
            nn.Linear(32, 10), 
            nn.LeakyReLU(0.1),
            nn.Softmax(dim=-1)  
        )
        
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x=self.dropout(x)
        x = self.linear_layers(x)
        return x


In [ ]:
CNNModel()

In [ ]:
model=CNNModel().to(device)
minibatch_size=128
optimizer= torch.optim.Adam(model.parameters(),lr=0.001)
loss_func=nn.CrossEntropyLoss()

nb_epochs=100
early_stop=10
progress_interval=5

In [ ]:
def train_model(model, early_stop, n_epochs, progress_interval):
    
    train_losses, valid_losses, lowest_loss = list(), list(), np.inf

   
    for epoch in range(n_epochs):
        
        train_loss, valid_loss = 0, 0
        
        model.train()
        
        for x_minibatch, y_minibatch in train_batches:
            x_minibatch = x_minibatch.to(device)
            y_minibatch = y_minibatch.to(device)
            y_minibatch_pred = model(x_minibatch)
            loss = loss_func(y_minibatch_pred, y_minibatch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss = train_loss / len(train_batches)
        train_losses.append(train_loss)      
        
    
        model.eval()
        with torch.no_grad():
            for x_minibatch, y_minibatch in val_batches:
                x_minibatch = x_minibatch.to(device)
                y_minibatch = y_minibatch.to(device)
                y_minibatch_pred = model(x_minibatch)
                loss = loss_func(y_minibatch_pred, y_minibatch)
                valid_loss += loss.item()
                
        valid_loss = valid_loss / len(val_batches)
        valid_losses.append(valid_loss)

        if valid_losses[-1] < lowest_loss:
            lowest_loss = valid_losses[-1]
            lowest_epoch = epoch
            best_model = deepcopy(model.state_dict())
        else:
            if (early_stop > 0) and lowest_epoch + early_stop < epoch:
                print ("Early Stopped", epoch, "epochs")
                break
                
        if (epoch % progress_interval) == 0:
            print (train_losses[-1], valid_losses[-1], lowest_loss, lowest_epoch, epoch)
            
    model.load_state_dict(best_model)        
    return model, lowest_loss, train_losses, valid_losses

In [ ]:
model, lowest_loss, trian_loss, valid_loss = train_model(model,early_stop,nb_epochs,progress_interval)

In [ ]:
test.shape
X_test=test

In [ ]:
X_test=X_test.values
X_test=X_test.reshape(-1,1,28,28)
X_test=torch.tensor(X_test, dtype=torch.float32)
test_batches=DataLoader(X_test,batch_size=minibatch_size,shuffle=True)

In [ ]:
X_test.shape

In [ ]:
model.eval()
all_pred=[]
with torch.no_grad():
    for x_minibatch in test_batches:
        x_minibatch=x_minibatch.to(device)
        y_test_pred = model(x_minibatch)
        all_pred.append(y_test_pred)
y_test_pred = torch.cat(all_pred, dim=0)

In [ ]:
y_test_pred.shape

In [ ]:
predicted_labels = torch.argmax(y_test_pred, dim=1)
display(predicted_labels)

In [ ]:
predicted_labels_numpy = predicted_labels.cpu().numpy()  # PyTorch 텐서를 NumPy 배열로 변환

submission_df = pd.DataFrame({'ImageId': range(1, len(predicted_labels_numpy) + 1),
                              'Label': predicted_labels_numpy})

submission_df.to_csv("submission.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(18 , 20))

for index in range(10):
    plt.subplot(10, 10, index + 1)
    plt.axis('off')
    plt.imshow(wrong_samples[index].numpy().reshape(28,28), cmap = "gray")
    plt.title("Pred" + str(wrong_preds[index].item()) + "(" + str(actual_preds[index].item()) + ")", color='red')

In [ ]:
test_loss = 0
correct = 0
wrong_samples, wrong_preds, actual_preds = list(), list(), list()

model.eval()
with torch.no_grad():
    for x_minibatch, y_minibatch in val_batches:
        x_minibatch = x_minibatch.to(device)
        y_minibatch = y_minibatch.to(device)      
        y_test_pred = model(x_minibatch)
        test_loss += loss_func(y_test_pred, y_minibatch)  
        pred = torch.argmax(y_test_pred, dim=1)
        correct += pred.eq(y_minibatch).sum().item()
        
        wrong_idx = pred.ne(y_minibatch).nonzero()[:, 0].cpu().numpy().tolist()
        for index in wrong_idx:
            wrong_samples.append(x_minibatch[index].cpu())
            wrong_preds.append(pred[index].cpu())
            actual_preds.append(y_minibatch[index].cpu())
            
test_loss /= len(test_batches.dataset)
print('Average Test Loss: {:.4f}'.format( test_loss ))
print('Accuracy: {}/{} ({:.2f}%)'.format( correct, len(test_batches.dataset), 100 * correct / len(test_batches.dataset) ))

In [ ]:
pds=pd.read_csv("/kaggle/working/submission.csv")
display(pds)